In [ ]:
# 필요한 라이브러리
from openai import OpenAI
import re
from google.cloud import vision
import os

In [ ]:
# [0] API 키 설정
client = OpenAI(api_key="your_key")  # 🔁 OpenAI API 키
# 🔁 Google Cloud Vision 인증 파일 설정
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "your_json"

# [1] OCR 추출 함수 (Google Cloud Vision 사용)
def detect_text(image_path):
    client = vision.ImageAnnotatorClient()
    with open(image_path, "rb") as image_file:
        content = image_file.read()
    image = vision.Image(content=content)
    response = client.text_detection(image=image)
    texts = response.text_annotations

    if response.error.message:
        raise Exception(
            f"{response.error.message}\nMore info: https://cloud.google.com/apis/design/errors"
        )
    return texts

# [2] GPT 요청: dict 형태로 추출
def ask_gpt_extract_dicts_from_full_text(full_text):
    prompt = f"""다음은 OCR로 추출한 영수증 전체 텍스트야.
여기서 결제 항목만 찾아서 다음과 같은 dict 형태 리스트로 정리해줘:
형식은 아래와 같아.
[
  {{"item": "상품명", "price": 가격(int), "category": ""}},
  ...
]

OCR 텍스트:
\"\"\"
{full_text}
\"\"\"
"""
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
    )
    return response.choices[0].message.content.strip()

In [ ]:
img_path = "img\\sample2.jpg"

texts = detect_text(img_path)
full_text = texts[0].description

print("\nOCR 전체 텍스트:\n")
print(full_text)


🧾 OCR 전체 텍스트:

농협
주소: 경기 의정부시
대표: 최영*
1:031-***-****
사업자번호: 127-82-*****
홈페이지 :http://www.nonghyup.com/
영수증 미지참시 교환/환불 불가 (30일 )
교환/환불 구매점에서 가능(결제카드지참)
김갑순
상품(코드)
2015-11-03 16:31:53
단가
수량
0002-00085
금액
001 P굿모닝 우유 900ML
[2.150]
+8801104210645
1.350
1
1.350
002 P양파
*231973
3.300
1
3,300
003 P.
*231913
500
1
500
004 P깻잎.
$231308
750
1
750
005 P하선정 바로먹기좋은장아찌
150g
8801007265889
1.380
1
1.380
006 P 브로커리.
$232285
1.280
1
1.280
판매총액 :
8,560
받을 금액:
8.560
신용 액
8.560
부가세면세물품가액:
부가세과세물품가액:
부 가 세:
7.180
1,255
125
바코드앞 면세. 영세. 상품명 P포인트
회원 : 2010190034*** 박분 님
우수고객포인트:
40
잔여포인트:
14.198
14.190
사용가능포인트:
****** 신용카드 매출전표(고객용) ******
219:4902************
*********
*********
할부 : 00 개월
매출금액:
8.560
승인 No: 75513401
승인일자 : 20151103
승 인
가맹점 : 00000074900
카드사 : KIS(V3)
좋은 하루 되세요!
언제나 고객님을 위해 최선을 다하겠습니다.
0111621511030002000850


In [ ]:
# 2. GPT에 전체 텍스트 전달
print("\n GPT dict 추출 결과:")
result = ask_gpt_extract_dicts_from_full_text(full_text)
print(result)


🤖 GPT dict 추출 결과:
[
    {"item": "P굿모닝 우유 900ML", "price": 1350, "category": ""},
    {"item": "P양파", "price": 3300, "category": ""},
    {"item": "P.", "price": 500, "category": ""},
    {"item": "P깻잎.", "price": 750, "category": ""},
    {"item": "P하선정 바로먹기좋은장아찌 150g", "price": 1380, "category": ""},
    {"item": "P 브로커리.", "price": 1280, "category": ""}
]


In [ ]:
import json

parsed_result = json.loads(result)

with open("gpt_receipt_result.json", "w", encoding="utf-8") as f:
    json.dump(parsed_result, f, ensure_ascii=False, indent=2)

In [19]:
with open("temptext.json", "w", encoding="utf-8") as f:
    json.dump(full_text, f, ensure_ascii=False, indent=2)